Googleドライブを使用できるようにするための準備

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

必要なライブラリ等の準備

In [ ]:
!pip install japanize-matplotlib

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import japanize_matplotlib 
import plotly
import plotly.graph_objs as go
# Google Colab. やJupyter Lab.でplotlyを使用するための準備
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default = "colab"
# orcaのインストール
!pip install plotly>=4.7.1
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

from plotly.offline import iplot, plot, download_plotlyjs, init_notebook_mode
import plotly.graph_objs as go

plt.rcParams['font.family']
plt.rcParams['font.family'] = 'IPAexGothic'

In [ ]:
!pip install plotly==5.3.1

In [ ]:
!pip install -U kaleido

In [ ]:
!pip install fpdf

# 「IPA」フォントをインストール
!apt-get -y install fonts-ipafont-gothic
# matplotlibのキャッシュをクリア
!rm /root/.cache/matplotlib/fontlist-v300.json

各データファイルの取得

In [ ]:
# ブラウザでフォルダを表示するとURLのfolders以降がフォルダのIDになっている
# https://drive.google.com/drive/folders/<ID>
dir_id = "(読み込みたいファイルがある場所)"

# フォルダ内にあるファイル名とファイルIDを表示する
file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % dir_id}).GetList()

file_dict = {}
for f in file_list:
  if(f["title"] in "data.csv"):
    file_dict["data.csv"] = f["id"]
  elif(f["title"] in "juyo-result-2020-j.csv"):
    file_dict["juyo-result-2020-j.csv"] = f["id"]
  else:
    file_dict["juyo-result-2019-j.csv"] = f["id"]

file_dict

In [ ]:
# 上記の方法で調べたファイルのID
file_id = file_dict["data.csv"]
drive_file = drive.CreateFile({'id': file_id})

# 平均気温ファイルの取得
drive_file.GetContentFile("data.csv")

data = pd.read_csv('data.csv', encoding="shift-jis", header=2)

2020年の電力実績データを取得し、月ごとの平均電力実績を求める

In [ ]:
file_id = file_dict["juyo-result-2020-j.csv"]
drive_file = drive.CreateFile({'id': file_id})

# 2020年の電力実績データファイルの取得
drive_file.GetContentFile("juyo-result-2020-j.csv")

juyo_result_2020_j = pd.read_csv('juyo-result-2020-j.csv', encoding="shift-jis", header=1)
jr2020 = juyo_result_2020_j.loc[:,["DATE", "実績(万kW)"]]
df_i = jr2020.set_index('DATE')

months = []
january = df_i.loc['2020/1/1':'2020/1/31']
months.append(january)
february = months.append(df_i.loc['2020/2/1':'2020/2/29'])
march = months.append(df_i.loc['2020/3/1':'2020/3/31'])
april = months.append(df_i.loc['2020/4/1':'2020/4/30'])
may = months.append(df_i.loc['2020/5/1':'2020/5/31'])
june = months.append(df_i.loc['2020/6/1':'2020/6/30'])
july = months.append(df_i.loc['2020/7/1':'2020/7/31'])
august = months.append(df_i.loc['2020/8/1':'2020/8/31'])
september = months.append(df_i.loc['2020/9/1':'2020/9/30'])
october = months.append(df_i.loc['2020/10/1':'2020/10/31'])
november = months.append(df_i.loc['2020/11/1':'2020/11/30'])
december = months.append(df_i.loc['2020/12/1':'2020/12/31'])

maa2020 = []
for month in months:
  month_achievement = month.loc[:,["実績(万kW)"]]
  maa2020.append(month_achievement.mean().values)

2019年の電力実績データを取得し、月ごとの平均電力実績を求める

In [ ]:
file_id = file_dict["juyo-result-2019-j.csv"]
drive_file = drive.CreateFile({'id': file_id})

# 2019年の電力実績データファイルの取得
drive_file.GetContentFile("juyo-result-2019-j.csv")

juyo_result_2019_j = pd.read_csv('juyo-result-2019-j.csv', encoding="shift-jis", header=1)
jr2019 = juyo_result_2019_j.loc[:,["DATE", "実績(万kW)"]]
df_i = jr2019.set_index('DATE')

months = []
january = months.append(df_i.loc['2019/1/1':'2019/1/31'])
february = months.append(df_i.loc['2019/2/1':'2019/2/28'])
march = months.append(df_i.loc['2019/3/1':'2019/3/31'])
april = months.append(df_i.loc['2019/4/1':'2019/4/30'])
may = months.append(df_i.loc['2019/5/1':'2019/5/31'])
june = months.append(df_i.loc['2019/6/1':'2019/6/30'])
july = months.append(df_i.loc['2019/7/1':'2019/7/31'])
august = months.append(df_i.loc['2019/8/1':'2019/8/31'])
september = months.append(df_i.loc['2019/9/1':'2019/9/30'])
october = months.append(df_i.loc['2019/10/1':'2019/10/31'])
november = months.append(df_i.loc['2019/11/1':'2019/11/30'])
december = months.append(df_i.loc['2019/12/1':'2019/12/31'])

maa2019 = []
for month in months:
  month_achievement = month.loc[:,["実績(万kW)"]]
  maa2019.append(month_achievement.mean().values)

# 2019年の電力実績の平均値と2020年の電力実績の平均値を一つのリストに格納する
maa = maa2019 + maa2020

In [ ]:
# 2019年と2020年の月ごとの平均電力実績をDataFrame化する
df = pd.DataFrame({'DATE': ['2019/1', '2019/2', '2019/3','2019/4', '2019/5', '2019/6','2019/7', '2019/8', '2019/9','2019/10', '2019/11', '2019/12', '2020/1', '2020/2', '2020/3','2020/4', '2020/5', '2020/6','2020/7', '2020/8', '2020/9','2020/10', '2020/11', '2020/12'],
                   '平均電力実績(万kW)': maa})
print(df["平均電力実績(万kW)"].dtype)

# グラフ化するために平均電力実績をfloat型に変換する
# 変数名eppはElectric power performanceの略
epp = df.astype({"平均電力実績(万kW)": float})
print(epp)

object
       DATE  平均電力実績(万kW)
0    2019/1  4273.967742
1    2019/2  4191.500000
2    2019/3  3672.741935
3    2019/4  3400.566667
4    2019/5  3286.354839
5    2019/6  3552.466667
6    2019/7  3978.064516
7    2019/8  4571.774194
8    2019/9  4076.933333
9   2019/10  3469.580645
10  2019/11  3535.333333
11  2019/12  3986.161290
12   2020/1  4119.903226
13   2020/2  3989.931034
14   2020/3  3671.354839
15   2020/4  3272.400000
16   2020/5  2971.774194
17   2020/6  3618.133333
18   2020/7  3794.258065
19   2020/8  4788.548387
20   2020/9  4026.266667
21  2020/10  3331.516129
22  2020/11  3462.233333
23  2020/12  4117.774194


2019年から2020年の電力実績をグラフ化する

In [ ]:
# 平均電力実績を表示するグラフを作成
trace1 = go.Bar(
            x=epp["DATE"],
            y=epp["平均電力実績(万kW)"],
            name ="電力実績可視化グラフ"
       )

data=[trace1]
go.Figure(data).show()

2019年から2020年の平均気温をグラフ化する

In [ ]:
data = pd.read_csv('data.csv', encoding="shift-jis", header=2)
temp = data.loc[1:,["年月", "平均気温(℃)"]]

trace2 = go.Line(
            x=temp["年月"],
            y=temp["平均気温(℃)"],
            name ="平均気温可視化グラフ"
       )
data=[trace2]

go.Figure(data).show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




平均気温のグラフと電力実績のグラフをマージする

In [ ]:
epp.rename(columns={"DATE": "年月"},
          inplace=True)
df = pd.merge(epp, temp, on="年月")
df.info()
print(df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   年月           24 non-null     object 
 1   平均電力実績(万kW)  24 non-null     float64
 2   平均気温(℃)      24 non-null     float64
dtypes: float64(2), object(1)
memory usage: 768.0+ bytes
         年月  平均電力実績(万kW)  平均気温(℃)
0    2019/1  4273.967742      5.6
1    2019/2  4191.500000      7.2
2    2019/3  3672.741935     10.6
3    2019/4  3400.566667     13.6
4    2019/5  3286.354839     20.0
5    2019/6  3552.466667     21.8
6    2019/7  3978.064516     24.1
7    2019/8  4571.774194     28.4
8    2019/9  4076.933333     25.1
9   2019/10  3469.580645     19.4
10  2019/11  3535.333333     13.1
11  2019/12  3986.161290      8.5
12   2020/1  4119.903226      7.1
13   2020/2  3989.931034      8.3
14   2020/3  3671.354839     10.7
15   2020/4  3272.400000     12.8
16   2020/5  2971.774194     19.5
17   2020/6  3618.1

東京の電力実績と平均気温の関係を表すグラフを作成する

In [ ]:
jp_font = {'fontname':'IPAGothic'} # 日本語フォント設定を作成

In [ ]:
import matplotlib.font_manager as font_manager

# 2つ目の軸を表示するか否かを設定
fig = make_subplots(specs=[[{"secondary_y": True}]])
# 第一軸のグラフ
fig.add_trace(go.Bar(
            x=df["年月"],
            y=df["平均電力実績(万kW)"],
            name ="電力実績(万kW)"
       ))
# 第二軸のグラフ
fig.add_trace(go.Line(
            x=df["年月"],
            y=df["平均気温(℃)"],
            name ="平均気温"),
       secondary_y=True)

data = [trace1, trace2]

layout = go.Layout(
    title="東京の電力実績と平均気温の関係"
)
fig.update_yaxes(title_text="<b>平均電力実績(万kW)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>平均気温(℃)</b>", secondary_y=True)
fig.update_xaxes(title="<b>年月</b>")

# このまま出力すると文字化けしてしまうので日本語のフォントを設定する
font = font_manager.FontProperties(
    family="IPAGothic",
    size=30
)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




作成したグラフをPDFファイルに出力する

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
fig.write_image("/content/drive/(置きたいディレクトリ)/東京の電力実績と平均気温の関係.pdf")

plotlyの利点は動的なグラフが作成できることにあるのでhtmlでも出力


In [ ]:
plot(fig, filename="/content/drive/(置きたいディレクトリ)/東京の電力実績と平均気温の関係.html", auto_open=False)

In [ ]:
fig.write_html('/content/drive/(置きたいディレクトリ)/東京の電力実績と平均気温の関係.html')